# Word segmentation of Lao bibliographic data

Install packages not available in Google Colab.

In [12]:
#!pip install laonlp
#!pip install pyicu

Import required packages.

In [13]:
import pandas as pd
import regex as re
from laonlp.tokenize import word_tokenize
from icu import BreakIterator, Locale

Sample Lao data from bib records. First column romanised Lao (ALA-LC Lao Romanisation Table 2012), second row is Original Lao script data in bib record (880 fields).

Create a pandas dataframe from bib data.

In [14]:
lao_data =[
    ["Kō̜n cha mī Mư̄ang Vīang Sai thān thīman kānpativat : bot banthưk khwāmsongcham / Somphō̜n Sīsuvanna.",  "ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດບັນທຶກຄວາມຊົງຈຳ / ສົມພອນ ສີສີສີນນະ."],
    ["Lom hāi chai khō̧ng phǣndin / Kom Khāosān Mǣnying Lāo Sūn Kāng Sahāphan Mǣnying Lāo.", "ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາງສະຫະພັນແມ່ມ່ງລາວ."],
    ["Sēnthāng sū santiphāp / khonkhwā læ hīaphīang, Suli Detvongphan.", "ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລິ ເດດວົງພັນ."]
]
source_df = pd.DataFrame(lao_data, columns = ['latin', 'lao'])
df = source_df
df

,latin,lao
0,Kō̜n cha mī Mư̄ang Vīang Sai thān thīman...,ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດ...
1,Lom hāi chai khō̧ng phǣndin / Kom Khāosān...,ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາ...
2,Sēnthāng sū santiphāp / khonkhwā læ hīap...,"ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລ..."


## Tokenisation with LaoNLP

_N.B. For the purposes of this exercise, I wish to return a string for comparison with the original Lao and transliterated Lao, rather than an iterator object or list._

LaoNLP provides a method to tokenise by word (`word_tokenize`) or by sentence (`sent_tokenize`). In this instance we the word tokeniser.

Create a segmentation fuunction to use with Pandas:

In [15]:
def laonlp_tokenise(s):
    s = " ".join(word_tokenize(s))
    s = re.sub(r"\s{2,}", " ", s)
    return re.sub(r'\s([?.!"](?:\s|$))', r'\1', s)

Create new column in dataframe with word segemented Lao script data:

In [16]:
df['laonlp'] = df['lao'].map(laonlp_tokenise)
df

,latin,lao,laonlp
0,Kō̜n cha mī Mư̄ang Vīang Sai thān thīman...,ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດ...,ກ່ອນຈະ ມີ ເມືອງ ວຽງໄຊ ຖານທີ່ ທີ່ ນ ການ ປະຕິ ຕິ...
1,Lom hāi chai khō̧ng phǣndin / Kom Khāosān...,ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາ...,ລົມຫາຍໃຈ ຂອງ ແຜ່ນດິນ / ກົມ ຂ່າວສານ ແມ່ຍິງ ລາວ ...
2,Sēnthāng sū santiphāp / khonkhwā læ hīap...,"ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລ...","ເສັ້ນທາງ ສູ່ ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ , ..."


In [17]:
# df['laonlp_latin_length_ne'] = df['laonlp'].str.split().str.len().ne(df['latin'].str.split().str.len())
# df


In [18]:
#laonlp.tokenize.lao_words()

## Word segmentation with PyICU

PyICU provides the following methods to create a break iterator: `createCharacterInstance`, `createCharacterInstance`, `createLineInstance`,
 `createSentenceInstance`, `createTitleInstance`, and `createWordInstance`. 
 
We will use `createWordInstance` to break at word boundaries.

Initially, we define a function to perform the segmentation of the string. LaoNLP provided a mechanism to do that directly. PyICU exposes lower level functions. This is more powerful and flexible, but requires the use to write functions to handle the segmentation of strings.

In [19]:
def iterate_breaks(text, bi):
    bi.setText(text)
    lastpos = 0
    while True:
        next_boundary = bi.nextBoundary()
        if next_boundary == -1: return
        yield text[lastpos:next_boundary]
        lastpos = next_boundary

def icu_tokenise(s):
    bi = BreakIterator.createWordInstance(Locale('lo_LA'))
    s = " ".join(list(iterate_breaks(s, bi)))
    s = re.sub(r"\s{2,}", " ", s)
    return re.sub(r'\s([?.!"](?:\s|$))', r'\1', s)

In [20]:
df['icu'] = df['lao'].map(icu_tokenise)
df

,latin,lao,laonlp,icu
0,Kō̜n cha mī Mư̄ang Vīang Sai thān thīman...,ກ່ອນຈະມີເມືອງວຽງໄຊ ຖານທີ່ທີ່ນ ການປະຕິຕິດ : ບົດ...,ກ່ອນຈະ ມີ ເມືອງ ວຽງໄຊ ຖານທີ່ ທີ່ ນ ການ ປະຕິ ຕິ...,ກ່ອນ ຈະ ມີ ເມືອງ ວຽງ ໄຊ ຖານ ທີ່ ທີ່ ນ ການ ປະຕິ...
1,Lom hāi chai khō̧ng phǣndin / Kom Khāosān...,ລົມຫາຍໃຈຂອງແຜ່ນດິນ / ກົມຂ່າວສານແມ່ຍິງລາວ ສູນກາ...,ລົມຫາຍໃຈ ຂອງ ແຜ່ນດິນ / ກົມ ຂ່າວສານ ແມ່ຍິງ ລາວ ...,ລົມ ຫາຍໃຈ ຂອງ ແຜ່ນດິນ / ກົມ ຂ່າວສານ ແມ່ ຍິງ ລາ...
2,Sēnthāng sū santiphāp / khonkhwā læ hīap...,"ເສັ້ນທາງສູ່ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ, ສຸລ...","ເສັ້ນທາງ ສູ່ ສັນຕິພາບ / ຄົ້ນຄວ້າ ແລະ ຮຽບຮຽງ , ...","ເສັ້ນທາງ ສູ່ ສັນຕິພາບ / ຄົ້ນ ຄວ້າ ແລະ ຮຽບຮຽງ ,..."


In [21]:
# df['icu_latin_length_ne'] = df['icu'].str.split().str.len().ne(df['latin'].str.split().str.len())
# df

In [22]:
# df['laonlp_icu_length_ne'] = df['laonlp'].str.split().str.len().ne(df['icu'].str.split().str.len())
# df